In [2]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.8f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [3]:
# %timeit sirve para evaluar el tiempo de ejecucion
events_test = pd.read_csv('test.csv')
events_training = pd.read_csv('training.csv')
#projects_2017.head()trocafone_kaggle_test
#projects_2017

In [4]:
test_con_feature  = pd.read_csv('train_paso_1.csv')
train_con_feature = pd.read_csv('train_paso_1.csv')

In [6]:
events_training
import calendar
events_training['timestamp'] = pd.to_datetime(events_training['timestamp'])
events_training['Anio'] = events_training['timestamp'].map(lambda x:x.year)
events_training['Mes'] = events_training['timestamp'].map(lambda x:x.month)
events_training['Dia'] = events_training['timestamp'].map(lambda x:x.dayofweek)
events_training['Mes_Nombre'] = events_training['timestamp'].map(lambda x:x.month)

events_training['Hora'] = pd.to_datetime(events_training['timestamp'],\
                                   format='%H:%M',errors='coerce').dt.hour

In [7]:
dia_semana=events_training.groupby(['person','Dia']).agg({'Dia':'count'}).unstack().fillna(0)
dia_semana.columns=['lunes','martes','miercoles','jueves','viernes','sabado','domingo']
dia_semana=dia_semana.reset_index()
dia_semana.head(5)

,person,lunes,martes,miercoles,jueves,viernes,sabado,domingo
0,0008ed71,0.00000000,0.00000000,0.00000000,6.00000000,0.00000000,0.00000000,0.00000000
1,000c79fe,0.00000000,17.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
2,001802e4,0.00000000,0.00000000,0.00000000,19.00000000,0.00000000,0.00000000,0.00000000
3,0019e639,122.00000000,142.00000000,67.00000000,21.00000000,80.00000000,9.00000000,30.00000000
4,001b0bf9,0.00000000,7.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000


In [8]:
 meses=events_training.groupby(['person','Mes']).agg({'Mes':'count'}).unstack().fillna(0)
meses.columns=['enero','febrero','marzo','abril','mayo']
meses=meses.reset_index()
meses.head(5)

,person,enero,febrero,marzo,abril,mayo
0,0008ed71,0.00000000,0.00000000,0.00000000,0.00000000,6.00000000
1,000c79fe,0.00000000,0.00000000,0.00000000,0.00000000,17.00000000
2,001802e4,0.00000000,0.00000000,0.00000000,0.00000000,19.00000000
3,0019e639,30.00000000,82.00000000,0.00000000,69.00000000,290.00000000
4,001b0bf9,0.00000000,0.00000000,0.00000000,0.00000000,7.00000000


In [9]:
events_training.head()

,Unnamed: 0,timestamp,event,person,url,sku,model,condition,storage,color,...,device_type,screen_resolution,operating_system_version,browser_version,label,Anio,Mes,Dia,Mes_Nombre,Hora
0,0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.00000000,iPhone 5s,Muito Bom,32GB,Cinza espacial,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0
1,1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.00000000,iPhone 5s,Muito Bom,64GB,Prateado,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0
2,2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,"5,907.00000000",iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0
3,3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,"6,023.00000000",iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0
4,4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,nan,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0


In [10]:
events_training['search_term'].isnull().sum()

1116112

In [11]:
cualidades = {"viewed product":["viewed product"],
       "brand listing":["brand listing"],
      "visited site":["visited site"],
       "ad campaign hit":["ad campaign hit"],
       "generic listing":["generic listing"],
        "searched products":["searched products"], 
         "search engine hit":["search engine hit"], 
          "checkout":["checkout"], 
          "staticpage":["staticpage"], 
          "conversion":["conversion"], 
          "lead":["lead"]      
                   

      }

In [12]:
 def propiedad_tiene(  propiedad):

    """Devuelve un 1 si la propiedad tiene alguna de las cualidades en 'cualidades', 0 si no."""
    
    return int(propiedad['checkout']*propiedad['conversion'])

In [13]:
def propiedad_tiene(cualidades, propiedad):

    """Devuelve un 1 si la propiedad tiene alguna de las cualidades en 'cualidades', 0 si no."""
    tiene_cualidad = False
    if not(pd.isnull(propiedad["event"])):
        descripcion = propiedad["event"]
        for cualidad in cualidades:
            tiene_cualidad = tiene_cualidad or (cualidad in propiedad["event"])
    return int(tiene_cualidad)

In [14]:
train_con_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414 entries, 0 to 19413
Data columns (total 18 columns):
person                       19414 non-null object
label                        19414 non-null int64
most_frequent_event          19414 non-null object
count_most_frequent_event    19414 non-null int64
less_frequent_event          19414 non-null object
count_less_frequent_event    19414 non-null int64
ad campaign hit              19414 non-null float64
brand listing                19414 non-null float64
checkout                     19414 non-null float64
conversion                   19414 non-null float64
generic listing              19414 non-null float64
lead                         19414 non-null float64
search engine hit            19414 non-null float64
searched products            19414 non-null float64
staticpage                   19414 non-null float64
viewed product               19414 non-null float64
visited site                 19414 non-null float64
mean_per_event  

In [15]:
train_con_feature.head()

,person,label,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,mean_per_event
0,0566e9c1,0,viewed product,23,checkout,1,6.00000000,3.00000000,1.00000000,1.00000000,15.00000000,0.00000000,1.00000000,0.00000000,1.00000000,23.00000000,17.00000000,7.55555556
1,6ec7ee77,0,ad campaign hit,1,ad campaign hit,1,1.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,1.00000000,1.00000000
2,abe7a2fb,0,viewed product,31,checkout,1,9.00000000,14.00000000,1.00000000,0.00000000,9.00000000,0.00000000,4.00000000,6.00000000,0.00000000,31.00000000,22.00000000,12.00000000
3,34728364,0,viewed product,24,search engine hit,1,0.00000000,5.00000000,0.00000000,0.00000000,3.00000000,0.00000000,1.00000000,0.00000000,0.00000000,24.00000000,4.00000000,7.40000000
4,87ed62de,0,viewed product,9,visited site,1,5.00000000,0.00000000,2.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,9.00000000,1.00000000,4.25000000


In [16]:
events_training.head()

,Unnamed: 0,timestamp,event,person,url,sku,model,condition,storage,color,...,device_type,screen_resolution,operating_system_version,browser_version,label,Anio,Mes,Dia,Mes_Nombre,Hora
0,0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.00000000,iPhone 5s,Muito Bom,32GB,Cinza espacial,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0
1,1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.00000000,iPhone 5s,Muito Bom,64GB,Prateado,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0
2,2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,"5,907.00000000",iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0
3,3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,"6,023.00000000",iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0
4,4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,nan,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,2018,5,4,5,0


In [17]:
channel=events_training.groupby(['person','channel']).agg({'channel':'count'}).unstack()
channel.columns=['Direct','Email','Organic','Paid','Referral','Social','desconocido']
channel=channel.reset_index()
channel=channel.fillna(0)
channel.head()

,person,Direct,Email,Organic,Paid,Referral,Social,desconocido
0,0008ed71,0.00000000,0.00000000,0.00000000,0.00000000,2.00000000,0.00000000,0.00000000
1,000c79fe,0.00000000,0.00000000,0.00000000,1.00000000,0.00000000,0.00000000,0.00000000
2,001802e4,0.00000000,0.00000000,0.00000000,1.00000000,0.00000000,0.00000000,0.00000000
3,0019e639,0.00000000,0.00000000,2.00000000,16.00000000,1.00000000,0.00000000,0.00000000
4,001ca5ee,2.00000000,0.00000000,2.00000000,5.00000000,6.00000000,0.00000000,0.00000000


In [18]:
events_training['date'] = pd.to_datetime(events_training['timestamp'])

In [19]:
storage=events_training.groupby(['person' ,'storage']).agg({'storage':'count'}).unstack().fillna(0)  
storage.head() 

storage                                            \
storage                 128GB                 16GB                256GB   
person                                                                    
0008ed71           0.00000000           0.00000000           0.00000000   
000c79fe           4.00000000           0.00000000           0.00000000   
001802e4           0.00000000           0.00000000           2.00000000   
0019e639           0.00000000          98.00000000           0.00000000   
001b0bf9           2.00000000           0.00000000           0.00000000   

                                                                         \
storage                  32GB                  4GB                512MB   
person                                                                    
0008ed71           2.00000000           0.00000000           0.00000000   
000c79fe           0.00000000           0.00000000           0.00000000   
001802e4           0.00000000           0.00000000           0.00000000   
0019e639          14.00000000           0.00000000           0.00000000   
001b0bf9           0.00000000           0.00000000           0.00000000   

                                                    
storage                  64GB                  8GB  
person                                              
0008ed71           1.00000000           0.00000000  
000c79fe           0.00000000           0.00000000  
001802e4           3.00000000           0.00000000  
0019e639           0.00000000          94.00000000  
001b0bf9           3.00000000           0.00000000

In [20]:
#Este metodo lo implementamos para poder encontrar la marca de cada producto
#La funcíon del mismo es recibir un objeto (que en este caso representa el producto 
#sobre el que actua el evento)
#y nos va a devolver la marca del producto

def hallarMarca(x):
   x=str(x)
   y=""
   if x.find('iPhone')==0:
       y="iPhone"
   elif x.find('Samsung')==0:
       y="Samsung"
   elif x.find('Motorola')==0:
       y="Motorola"
   elif x.find('iPad')==0:
       y="iPad"
   elif x.find('LG')==0:
       y="LG"
   elif x.find('Quantum')==0:
       y="Quantum"
   elif x.find('Blackberry')==0:
       y="Blackberry"
   elif x.find('Asus')==0:
       y="Asus"
   elif x.find('Lenovo')==0:
       y="Lenovo"
   elif x.find('Sony')==0:
       y="Sony"
   return y


In [21]:
#Aplico la funcíon hallarMarca a cada elemento de la columna 'models'
#El resultado es una serie con la marca de cada producto (en orden)

sMarcas=events_training.apply({'model':hallarMarca})
#Creo un nuevo dataframe con la columna 'mark', que representa las marcas de cada producto

dataframeConMarcas=events_training
dataframeConMarcas['mark']=sMarcas
dataframeConMarcas.head()
# grafico=conversiones['mark'].value_counts().plot(kind='barh',figsize=(10,6),title='Marca \

,Unnamed: 0,timestamp,event,person,url,sku,model,condition,storage,color,...,operating_system_version,browser_version,label,Anio,Mes,Dia,Mes_Nombre,Hora,date,mark
0,0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.00000000,iPhone 5s,Muito Bom,32GB,Cinza espacial,...,NaN,NaN,0,2018,5,4,5,0,2018-05-18 00:11:27,iPhone
1,1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.00000000,iPhone 5s,Muito Bom,64GB,Prateado,...,NaN,NaN,0,2018,5,4,5,0,2018-05-18 00:23:33,iPhone
2,2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,"5,907.00000000",iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,...,NaN,NaN,0,2018,5,4,5,0,2018-05-18 00:16:10,iPhone
3,3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,"6,023.00000000",iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,...,NaN,NaN,0,2018,5,4,5,0,2018-05-18 00:14:55,iPhone
4,4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,nan,NaN,NaN,NaN,NaN,...,NaN,NaN,0,2018,5,4,5,0,2018-05-18 00:11:26,


In [22]:
datatrain=pd.merge(train_con_feature,storage,on='person',how='left')
datatrain=datatrain.fillna(0)
datatrain.isnull().sum()

person                       0
label                        0
most_frequent_event          0
count_most_frequent_event    0
less_frequent_event          0
count_less_frequent_event    0
ad campaign hit              0
brand listing                0
checkout                     0
conversion                   0
generic listing              0
lead                         0
search engine hit            0
searched products            0
staticpage                   0
viewed product               0
visited site                 0
mean_per_event               0
(storage, 128GB)             0
(storage, 16GB)              0
(storage, 256GB)             0
(storage, 32GB)              0
(storage, 4GB)               0
(storage, 512MB)             0
(storage, 64GB)              0
(storage, 8GB)               0
dtype: int64

In [23]:
storage=events_training.groupby(['person' ,'storage']).agg({'storage':'count'})  
print (len(events_training))

1171886


In [24]:
def agregarcolumnas(archivo,archivo_con_feature):
    #archivo_con_feature es un archivo test o train con feature antes calculada
    #archivo es el train con label o el test
    
     
    archivo['timestamp'] = pd.to_datetime(archivo['timestamp'])
    archivo['Anio'] = archivo['timestamp'].map(lambda x:x.year)
    archivo['Mes'] = archivo['timestamp'].map(lambda x:x.month)
    archivo['Dia'] = archivo['timestamp'].map(lambda x:x.dayofweek)
    archivo['Mes_Nombre'] = archivo['timestamp'].map(lambda x:x.month)

    archivo['Hora'] = pd.to_datetime(archivo['timestamp'],\
                                   format='%H:%M',errors='coerce').dt.hour
    #agrego divicion entre convert y checkout
    
    #cantidad_eventos=archivo.groupby(['person','event']).agg({'event':'count'})\
    #.unstack().fillna(0)
    #cantidad_eventos.columns=['ad campaign hit','brand listing','checkout','conversion','generic listing','lead','search engine hit','searched products','staticpage','viewed product','visited site']
    #cantidad_eventos=cantidad_eventos.reset_index()
    #cantidad_eventos['div_conver_check']=cantidad_eventos['conversion'].astype(float)/\
    #cantidad_eventos['checkout'].astype(float)
    #cantidad_eventos=cantidad_eventos.fillna(0)
    #cantidad_eventos=cantidad_eventos[['person','div_conver_check']]
    #cantidad_eventos.head()
    
    #agrego storage
    print (len(archivo))
    storage=archivo.groupby(['person' ,'storage']).agg({'storage':'count'}).unstack().fillna(0) 
    print (len(storage))
    storage.columns=['128GB','16GB','256GB','32GB','4GB','512MB','64GB','8GB']
    print (len(storage))
    storage=storage.reset_index() 
    print (len(storage))
    storage.head(5)
    print (len(storage))
    archivo_con_storage=pd.merge(archivo_con_feature,storage,on='person',how='left')
    archivo_con_storage=archivo_con_storage.fillna(0)
    print (len(archivo_con_storage))
    #agrego condition
    
    condition=archivo.groupby(['person' ,'condition']).agg({'condition':'count'}).unstack().fillna(0) 
  
    condition.columns=['Bom','Bom - Sem Touch ID','Excelente','Muito Bom','Novo']
    condition=condition.reset_index() 
    condition.head(5)
    archivo_con_storage_condition=pd.merge(archivo_con_storage,condition,on='person',how='left')
    archivo_con_storage_condition=archivo_con_storage_condition.fillna(0)
    print (len(archivo_con_storage_condition))
    
    #agrego columnas modelo
    
    #Aplico la funcíon hallarMarca a cada elemento de la columna 'models'
    #El resultado es una serie con la marca de cada producto (en orden)

    sMarcas=archivo.apply({'model':hallarMarca})
    #Creo un nuevo dataframe con la columna 'mark', que representa las marcas de cada producto

    dataframeConMarcas=archivo
    dataframeConMarcas['mark']=sMarcas
    dataframeConMarcas.head()
    a=dataframeConMarcas['event']=='viewed product'
    b=dataframeConMarcas['event']=='checkout'
    c=dataframeConMarcas['event']=='conversion'
    data_mark=dataframeConMarcas.groupby(['person','mark']).agg({'mark':'count'}).unstack().fillna(0)
    data_mark.columns=['desconocido_modelo','Asus','LG','Lenovo','Motorola','Quantum','Samsung','Sony','iPad','iPhone']
    data_mark=data_mark.reset_index()
    archivo_con_storage_condition_mark=pd.merge(archivo_con_storage_condition,data_mark,on='person',how='left')
    archivo_con_storage_condition_mark=archivo_con_storage_condition_mark.fillna(0)
    
    #agrego las columnas channel
    
    
    #channel=archivo.groupby(['person','channel']).agg({'channel':'count'}).unstack()
    #channel.columns=['Direct','Email','Organic','Paid','Referral','Social','desconocido_channel']
    #channel=channel.reset_index()
    #channel=channel.fillna(0)
    #archivo_con_storage_condition_mark_cha=pd.merge(archivo_con_storage_condition_mark,channel,on='person',how='left')
    #archivo_con_storage_condition_mark_cha=archivo_con_storage_condition_mark_cha.fillna(0)
    
    
    #agrego suma y multiplicacion entre columnas conversion y checkout
    archivo_con_storage_condition_mark['multiplicacion_checkout_y_conversion'] = archivo_con_storage_condition_mark.apply(lambda row:   row['checkout']*row['conversion'], axis = 1)
    archivo_con_storage_condition_mark['suma_checkout_y_conversion'] = archivo_con_storage_condition_mark.apply(lambda row:   row['checkout']+row['conversion'], axis = 1)
    
    #dia de la semana con la cantidad que eventos
    dia_semana=archivo.groupby(['person','Dia']).agg({'Dia':'count'}).unstack().fillna(0)
    dia_semana.columns=['lunes','martes','miercoles','jueves','viernes','sabado','domingo']
    dia_semana=dia_semana.reset_index()
    dia_semana.head(5)
    
    archivo_con_storage_condition_mark_dia=pd.merge(archivo_con_storage_condition_mark,dia_semana,on='person',how='left')
    archivo_con_storage_condition_mark_dia=archivo_con_storage_condition_mark_dia.fillna(0)
    
    #dia del mes con mas eventos
    meses=archivo.groupby(['person','Mes']).agg({'Mes':'count'}).unstack().fillna(0)
    meses.columns=['enero','febrero','marzo','abril','mayo']
    meses=meses.reset_index()
    meses.head(5)
    archivo_con_storage_condition_mark_dia_mes=pd.merge(archivo_con_storage_condition_mark_dia,meses,on='person',how='left')
    archivo_con_storage_condition_mark_dia_mes=archivo_con_storage_condition_mark_dia_mes.fillna(0)
    
    
    return archivo_con_storage_condition_mark_dia_mes 
    
    

In [25]:
data_train=agregarcolumnas(events_training,train_con_feature)
data_train.info()
  
    

1171886
19128
19128
19128
19128
19414
19414
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 55 columns):
person                                  19414 non-null object
label                                   19414 non-null int64
most_frequent_event                     19414 non-null object
count_most_frequent_event               19414 non-null int64
less_frequent_event                     19414 non-null object
count_less_frequent_event               19414 non-null int64
ad campaign hit                         19414 non-null float64
brand listing                           19414 non-null float64
checkout                                19414 non-null float64
conversion                              19414 non-null float64
generic listing                         19414 non-null float64
lead                                    19414 non-null float64
search engine hit                       19414 non-null float64
searched products                    

In [26]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 55 columns):
person                                  19414 non-null object
label                                   19414 non-null int64
most_frequent_event                     19414 non-null object
count_most_frequent_event               19414 non-null int64
less_frequent_event                     19414 non-null object
count_less_frequent_event               19414 non-null int64
ad campaign hit                         19414 non-null float64
brand listing                           19414 non-null float64
checkout                                19414 non-null float64
conversion                              19414 non-null float64
generic listing                         19414 non-null float64
lead                                    19414 non-null float64
search engine hit                       19414 non-null float64
searched products                       19414 non-null float64
staticpage        

In [27]:
data_test=agregarcolumnas(events_test,test_con_feature)
data_test.head() 

1169795
19122
19122
19122
19122
19414
19414


,person,label,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,...,miercoles,jueves,viernes,sabado,domingo,enero,febrero,marzo,abril,mayo
0,0566e9c1,0,viewed product,23,checkout,1,6.00000000,3.00000000,1.00000000,1.00000000,...,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
1,6ec7ee77,0,ad campaign hit,1,ad campaign hit,1,1.00000000,0.00000000,0.00000000,0.00000000,...,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
2,abe7a2fb,0,viewed product,31,checkout,1,9.00000000,14.00000000,1.00000000,0.00000000,...,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
3,34728364,0,viewed product,24,search engine hit,1,0.00000000,5.00000000,0.00000000,0.00000000,...,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
4,87ed62de,0,viewed product,9,visited site,1,5.00000000,0.00000000,2.00000000,0.00000000,...,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000


In [28]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 55 columns):
person                                  19414 non-null object
label                                   19414 non-null int64
most_frequent_event                     19414 non-null object
count_most_frequent_event               19414 non-null int64
less_frequent_event                     19414 non-null object
count_less_frequent_event               19414 non-null int64
ad campaign hit                         19414 non-null float64
brand listing                           19414 non-null float64
checkout                                19414 non-null float64
conversion                              19414 non-null float64
generic listing                         19414 non-null float64
lead                                    19414 non-null float64
search engine hit                       19414 non-null float64
searched products                       19414 non-null float64
staticpage        

In [29]:
#agrega columnas test

In [30]:
data_train.to_csv("train_paso_2.csv", index = False)

In [31]:
data_test.to_csv("test_paso_2.csv", index = False)

In [32]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 55 columns):
person                                  19414 non-null object
label                                   19414 non-null int64
most_frequent_event                     19414 non-null object
count_most_frequent_event               19414 non-null int64
less_frequent_event                     19414 non-null object
count_less_frequent_event               19414 non-null int64
ad campaign hit                         19414 non-null float64
brand listing                           19414 non-null float64
checkout                                19414 non-null float64
conversion                              19414 non-null float64
generic listing                         19414 non-null float64
lead                                    19414 non-null float64
search engine hit                       19414 non-null float64
searched products                       19414 non-null float64
staticpage        